Some resources for training and testing NN models with R and the neuralnet package

https://rpubs.com/ksevong/392295 

https://www.datacamp.com/community/tutorials/neural-network-models-r

https://cran.r-project.org/web/packages/neuralnet/neuralnet.pdf







In [ ]:
# Upload the 'data1.csv' data file from Module 7 data folder

<font color='blue'>Cell 1 - Load packages and Iris data set


In [ ]:
## load all necessary packages :
install.packages("neuralnet")
install.packages('dplyr')
library(neuralnet)
library(dplyr)
 


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘Deriv’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)


Attaching package: ‘dplyr’


The following object is masked from ‘package:neuralnet’:

    compute


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [ ]:
## Retrieve iris data set

# Upload 'data1.csv' file from the either the drive, or your folder. 
# 1) Click the folder icon on the left side of the screen.
# 2) Click the 'upload to session' icon
# 3) locate the 'data1.csv' file and upload it.

iris <- read.csv('data1.csv')
summary(iris)
head(iris)

  Sepal_Length    Sepal_Width     Petal_Length    Petal_Width   
 Min.   :4.300   Min.   :2.000   Min.   :1.000   Min.   :0.100  
 1st Qu.:5.100   1st Qu.:2.800   1st Qu.:1.600   1st Qu.:0.300  
 Median :5.800   Median :3.000   Median :4.350   Median :1.300  
 Mean   :5.843   Mean   :3.054   Mean   :3.759   Mean   :1.199  
 3rd Qu.:6.400   3rd Qu.:3.300   3rd Qu.:5.100   3rd Qu.:1.800  
 Max.   :7.900   Max.   :4.400   Max.   :6.900   Max.   :2.500  
   Species         
 Length:150        
 Class :character  
 Mode  :character  
                   
                   
                   

,Sepal_Length,Sepal_Width,Petal_Length,Petal_Width,Species
,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,5.1,3.5,1.4,0.2,Iris-setosa
2,4.9,3.0,1.4,0.2,Iris-setosa
3,4.7,3.2,1.3,0.2,Iris-setosa
4,4.6,3.1,1.5,0.2,Iris-setosa
5,5.0,3.6,1.4,0.2,Iris-setosa
6,5.4,3.9,1.7,0.4,Iris-setosa


<font color='blue'>Cell 2 - Normalize the data

In [ ]:
## Normalize the data
# Save a copy of the original data set
iris_org <- iris

# Function that normalizes the iris data set 
normalizeData <- function(data){
        # Uses scale function to normalize data, with mean = 0, and sd = 1
        norm <- data.frame(scale(data[,1:4]))
        # Adds back on the Species column
        new_data <- cbind(norm, data$Species)
        # Renames the 5th column
        names(new_data)[5] <- 'Species'
        # Returns the data
        return(new_data)
} 
# Call upon the normalizeData function
iris <- normalizeData(iris)
# Shows the first 6 rows of the new iris data set.
head(iris)

,Sepal_Length,Sepal_Width,Petal_Length,Petal_Width,Species
,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,-0.8976739,1.0286113,-1.336794,-1.308593,Iris-setosa
2,-1.1392005,-0.1245404,-1.336794,-1.308593,Iris-setosa
3,-1.3807271,0.3367203,-1.393470,-1.308593,Iris-setosa
4,-1.5014904,0.1060900,-1.280118,-1.308593,Iris-setosa
5,-1.0184372,1.2592416,-1.336794,-1.308593,Iris-setosa
6,-0.5353840,1.9511326,-1.166767,-1.046525,Iris-setosa


<font color='blue'>Cell 3 - Randomize and split data

In [ ]:
## Randomize and split the data into a training set and a test set
## in a 2:1 ratio, respectively.
train_idx <- sample(nrow(iris), 2/3 * nrow(iris))
iris_train <- iris[train_idx, ]
iris_test <- iris[-train_idx, ]

head(iris_train)
cat("Training Data Dimensions: ", dim(iris_train))

head(iris_test)
cat("Testing Data Dimensions: ",dim(iris_test))

,Sepal_Length,Sepal_Width,Petal_Length,Petal_Width,Species
,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
104,0.5514857,-0.3551707,1.0435910,0.7879511,Iris-virginica
116,0.6722490,0.3367203,0.8735635,1.4431211,Iris-virginica
50,-1.0184372,0.5673506,-1.3367940,-1.3085928,Iris-setosa
107,-1.1392005,-1.2776920,0.4201569,0.6569171,Iris-virginica
73,0.5514857,-1.2776920,0.6468602,0.3948491,Iris-versicolor
66,1.0345390,0.1060900,0.3634810,0.2638151,Iris-versicolor


Training Data Dimensions:  100 5

,Sepal_Length,Sepal_Width,Petal_Length,Petal_Width,Species
,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
2,-1.1392005,-0.1245404,-1.336794,-1.308593,Iris-setosa
3,-1.3807271,0.3367203,-1.393470,-1.308593,Iris-setosa
5,-1.0184372,1.2592416,-1.336794,-1.308593,Iris-setosa
6,-0.5353840,1.9511326,-1.166767,-1.046525,Iris-setosa
17,-0.5353840,1.9511326,-1.393470,-1.046525,Iris-setosa
18,-0.8976739,1.0286113,-1.336794,-1.177559,Iris-setosa


Testing Data Dimensions:  50 5

<font color='blue'>Cell 4  - Create the neural network

In [ ]:
#Creating the neural network
#The first argument specifies the output classes (setosa+versicolor+virginica) and (~) the input features (Sepal.Length+Sepal.Width+Petal.Length+Petal.Width)
#The second argument specifies the data frame from which these inputs and outputs are taken (training data)
#The third argument specifies the hidden layer(s) using a list. Here a single hidden layer with 3 units is used 
nn <- neuralnet((Species == "Iris-setosa") + (Species == "Iris-versicolor") + (Species == "Iris-virginica")
                ~ Sepal_Length+Sepal_Width+Petal_Length + Petal_Width, iris_train,hidden = c(3), linear.output = FALSE)
# Show plot of the neural network
plot(nn)


<font color='blue'>Cell 5 - Predict using the Neural Network

In [ ]:
#Running the prediction function on the test set

pred <- predict(nn, iris_test)

head(pred)
# notice in each row how one value is 1, or very close to 1, 
# while the other two are very small/approach to zero

2,1,1.396980e-13,2.367015e-61
3,1,1.396987e-13,2.367003e-61
5,1,1.396994e-13,2.366992e-61
6,1,1.396990e-13,2.366998e-61
17,1,1.396995e-13,2.366990e-61
18,1,1.396990e-13,2.366998e-61


<font color='blue'>Cell 7 - Create a Confusion Matrix
  

In [ ]:
# Creating the confusion matrix to compare the predicted results with the observed test data

# Function that takes two arguments, the predicted results, and the actual 
# test results and produces a confusion matrix
createCM <- function(pred_data,observed_values){
        predicted_values <- apply(pred_data, 1, which.max)
        tab <- table(predicted_values,observed_values)
        rownames(tab) <- c('Iris-setosa','Iris-sersicolor', 'Iris-virginica')
        return(tab)
}
#  Call upon CreateCM function
cm <- createCM(pred,iris_test$Species)
# print out Confusion matrix
cm
# normalize the confusion matrix
normalizeCM <- function(cm){
        cm_norm <- cm
        for ( i in 1:3){
                for ( j in 1:3){
                        cm_norm[i,j] <- cm[i,j]/sum(cm[i,])
                }
        }
        
        return(round(cm_norm, 2))
}
# call upon the normalizeCM function and print out the normalized matrix
normalizeCM(cm)

                 observed_values
predicted_values  Iris-setosa Iris-versicolor Iris-virginica
  Iris-setosa              19               0              0
  Iris-sersicolor           0              14              1
  Iris-virginica            0               1             15

                 observed_values
predicted_values  Iris-setosa Iris-versicolor Iris-virginica
  Iris-setosa            1.00            0.00           0.00
  Iris-sersicolor        0.00            0.93           0.07
  Iris-virginica         0.00            0.06           0.94

<font color='blue'>Cell 8

In [ ]:
# Function to Normalize new input data
normalizeNewValues <- function(features){
        norm_features <- data.frame('Sepal_Length'= 0, 'Sepal_Width'= 0,
                                       'Petal_Length'= 0, 'Petal_Width'= 0)
        for (i in 1:4){
                x <- ((features[i]-mean(iris_org[,i]))/sd(iris_org[,i]))
                norm_features[1,i] <- x
                
        } 
        return(norm_features)
}

<font color='blue'>Cell 9- Test the Neural Network - Define the InputValue function

In [ ]:
## Testing the Neural Network with user inputted values
# The below function inputValue asks the user to enter a numeric value for a certain feature
inputValue <- function(feature_name){
  # creates message to prompt user to input the respective value
  message <- paste("Enter ", feature_name, ": ")
  while(TRUE){
    # If below zero or above 10, prints out error message, and prompts the user once more 
    # to input a value for the respective feature
    value = as.double(readline(prompt = message))
    if(value<0 | value>10){
      warning("Invalid entry, please try again")
    } else {
      
      return(value)
    }
  } 
}

<font color='blue'>Cell 10- Define the predictSpecies function

In [ ]:
# Create a species level vector to transpose numeric values of 1,2,3 into 
# 'iris-setosa', 'iris-versicolor', and 'iris-virginica', respectively
species_levels <- levels(factor(iris$Species))

## Function to predict species 
predictSpecies <- function(measurements){
        # Call upon the  normalizeNewValues function
        norm_measurements <- normalizeNewValues(measurements)
        # Using the predict function and our created neural network, create prediction 
        # probabilities for the 3 possible outcomes 
        prediction_prob <- predict(nn, norm_measurements)
        # Select the most probably species. 
        prediction <- which.max(prediction_prob)
        # Change prediction from numeric to character using species_levels
        prediction <- species_levels[prediction]
        # Inform user of calculated prediction
        cat("That flower is most likely: ", prediction)
}



<font color='blue'>Cell 11

In [ ]:
# Call upon the inputValue function four times to obtain the four input values
print('Measurements need to be numeric values between 0 and 10')
sepal_length <-  inputValue('Sepal Length')
sepal_width <-  inputValue('Sepal Width')
petal_length <- inputValue('Petal Length')
petal_width <- inputValue('Petal Width')

# create a list with the four given measurements
features = c(sepal_length, sepal_width, petal_length, petal_width)

# Call upon the predictSpecies function 
predictSpecies(features)

[1] "Measurements need to be numeric values between 0 and 10"
Enter  Sepal Length : 5.1
Enter  Sepal Width : 3.4
Enter  Petal Length : 1.4
Enter  Petal Width : .2
That flower is most likely:  Iris-setosa